In [23]:
import numpy as np
import cv2
import pandas as pd

np.random.seed(42)
import numpy as np
import cv2
import pandas as pd
import imghdr

np.random.seed(42)
class Yolo_BBox_Visualizer:

    def __init__(self):
        pass

    def read_txt(self, path_to_txt):
        df = pd.read_csv(f"{path_to_txt}",
                         sep=" ",
                         names=["label", "x", "y", "w", "h"])
        return df

    def read_classes(self, classes_file_path, random=True):
        with open(classes_file_path, "r") as f:
            self.classes_list = f.read().splitlines()
        # Random Colors list
        if random:
            self.color_list = np.random.uniform(
                low=0,
                high=255,
                size=(len(self.classes_list), 3)
                )
        # Green color only
        else:
            classes = np.zeros(len(self.classes_list))
            self.color_list = [(0,255,0) for i in classes]


    def convert(self, size, x, y, w, h):
        """
        From https://github.com/adi1999/yolo-bbox-visualizer/blob/master/main.py
        Convert bounding box coordinates
        size: tuple representing the width and height of an image
        x: x-coordinate of the center of a bounding box
        y: y-coordinate of the center of a bounding box
        w: width of the bounding box in fractional representation
        h: height of the bounding box
        return 
        """
        box = np.zeros(4)
        dw = 1. / size[0]
        dh = 1. / size[1]
        x = x / dw
        w = w / dw
        y = y / dh
        h = h / dh
        box[0] = x - (w / 2.0)
        box[1] = x + (w / 2.0)
        box[2] = y - (h / 2.0)
        box[3] = y + (h / 2.0)

        return (box)

    def draw_bbox(self, image, df):
        for i, row in df.iterrows():
            class_color = self.color_list[int(row["label"])]
            x, y, w, h =row['x'], row['y'], row['w'], row['h']
            img_h, img_w = image.shape[:2]
            bb = self.convert((img_w, img_h), x, y, w, h)
            cv2.rectangle(image, (int(round(bb[0])), int(round(bb[2]))),
                          (int(round(bb[1])), int(round(bb[3]))), class_color,
                          1)
        return image

    def draw_image(self, image_path, path_to_txt, img_name = "Image.jpg"):
        image = cv2.imread(image_path)
        bbox_image = self.draw_bbox(image, self.read_txt(path_to_txt))

        if image is None:
            print("Error: Failed to read image from file")
            return
        try:
            cv2.imwrite(img_name, bbox_image)
            cv2.imshow("Result", bbox_image)
            cv2.waitKey(0)

            cv2.destroyAllWindows()
        except:
            print("Error")

    
    def preprocess_image(self, image_path, model_image_size):
        image_type = imghdr.what(image_path)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        resized_image = cv2.resize(image, tuple(reversed(model_image_size)), interpolation=cv2.INTER_CUBIC)
        image_data = np.array(resized_image, dtype='float32')
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.
        return image, image_data
    
    def read_anchors(self, annotation_path):
        with open(annotation_path) as f:
            annotations = f.readlines()
    
        all_boxes = []
        for annotation in annotations:
            boxes = annotation.strip().split()[1:]
            boxes = np.array(boxes).reshape(-1, 2)
            all_boxes.append(boxes)
        return all_boxes

In [24]:
img_path = 'img.jpg'
label_path ='label.txt'

In [25]:
class_file = "class.names"

est = Yolo_BBox_Visualizer()
est.read_classes(class_file, random=True)
est.draw_image(img_path, label_path)

[152.65791347  39.78475331  39.77860269]
0
[ 95.50773031 242.43214813 186.65845516]
1
[152.65791347  39.78475331  39.77860269]
2
[152.65791347  39.78475331  39.77860269]
3
[152.65791347  39.78475331  39.77860269]
4
[152.65791347  39.78475331  39.77860269]
5
[ 95.50773031 242.43214813 186.65845516]
6
[ 95.50773031 242.43214813 186.65845516]
7
[152.65791347  39.78475331  39.77860269]
8
[152.65791347  39.78475331  39.77860269]
9
[152.65791347  39.78475331  39.77860269]
10
[152.65791347  39.78475331  39.77860269]
11
[152.65791347  39.78475331  39.77860269]
12


In [9]:
df = est.read_txt(label_path)

In [16]:
for index, row in df.iterrows():
    print(int(row['label']))
#     print(row['label'], row['x'])
#     print(index)

1
0
1
1
1
1
0
0
1
1
1
1
1


In [10]:
row['label']

,label,x,y,w,h
0,1,0.06500,0.701394,0.1210,0.303742
1,0,0.42225,0.063830,0.0405,0.066031
2,1,0.32050,0.059061,0.0580,0.069699
3,1,0.05250,0.020176,0.0540,0.040352
4,1,0.25700,0.172781,0.0540,0.081438
5,1,0.34550,0.184519,0.0680,0.100514
6,0,0.58700,0.206897,0.0410,0.070433
7,0,0.54950,0.263756,0.0600,0.084373
8,1,0.68675,0.142700,0.0565,0.094644
9,1,0.82450,0.195891,0.0470,0.080704
